In [1]:
import pyodbc

import pandas as pd

In [2]:
server = "SDDBSBI002"
database = "DSSG"

db_string = "DRIVER={ODBC Driver 13 for SQL Server};" + \
            "Trusted_connection=yes;" + \
            f"SERVER={server};" + \
            f"DATABASE={database}"

connection = pyodbc.connect(db_string)

In [2]:
results_sql = """
SELECT *
FROM lab.dim_test_result_output_v1 AS table_0
WHERE result_full_description IS NOT NULL
    AND result_full_description <> ''
    AND ISNUMERIC(result_full_description) <> 1
    AND NOT EXISTS (
        SELECT DISTINCT DTR.test_key, DTR.result_key
        FROM lab.dim_test_result DTR, lab.brg_result BR, lab.dim_result_hub DRH
        WHERE DTR.result_key = BR.result_key
            AND BR.result_hub_key = DRH.result_hub_key
            AND (DRH.result_code = 'PROF'
                OR DRH.result_code = 'PROTR'
                OR DRH.result_description LIKE '%proficiency%')
            AND DTR.test_key = table_0.test_key
            AND DTR.result_key = table_0.result_key
    )
"""

In [3]:
df_results = pd.read_sql(results_sql, connection)
df_results.to_csv("../data/dataset.csv", na_rep="_NULL_", index=False)

In [3]:
metamap_sql = """
SELECT *
FROM dbo.metamap
"""

In [4]:
df_metamap = pd.read_sql(metamap_sql, connection)
df_metamap.to_csv("../data/metamap.csv", na_rep="_NULL_", index=False)

In [5]:
connection.close()